In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
from configfile import db_user, db_password, db_host, db_port, db_name

In [2]:
# Files to Load
file1 = "resources/steam-200k.csv"
file2 = "resources/vgsales-12-4-2019-short.csv"

# Read and store into Pandas data frame
steam_data = pd.read_csv(file1)
sales_data = pd.read_csv(file2)

In [3]:
#Data Cleaning
steam_data=steam_data.dropna(how="any").drop(columns=["Unnamed: 4"])
steam_data=steam_data.loc[(steam_data["pur_play"])=="play",:]
steam_data=steam_data.drop(columns=["pur_play"]).rename(columns={"stat": "Hours"})    
steam_data

UserId                        Game  Hours
1       151603712  The Elder Scrolls V Skyrim  273.0
3       151603712                   Fallout 4   87.0
5       151603712                       Spore   14.9
7       151603712           Fallout New Vegas   12.1
9       151603712               Left 4 Dead 2    8.9
...           ...                         ...    ...
199991  128470551                Fallen Earth    2.4
199993  128470551                 Magic Duels    2.2
199995  128470551                 Titan Souls    1.5
199997  128470551  Grand Theft Auto Vice City    1.5
199999  128470551                        RUSH    1.4

[70489 rows x 3 columns]

In [4]:
# Grouping by game, number of players, total hours played, 
# and percent of users that play the game (of the dataset sample size)
group=steam_data.groupby("Game")
group_steam=pd.DataFrame({"Players":group.UserId.count(),"Hours":group.Hours.sum()})
group_steam["Percent Play"]=group_steam["Players"]/len(steam_data.UserId.value_counts())
group_steam

Players  Hours  \
Game                                                                 
007 Legends                                               1    0.7   
0RBITALIS                                                 3    1.2   
1... 2... 3... KICK IT! (Drop That Beat Like an...        5   20.0   
10 Second Ninja                                           2    5.9   
10,000,000                                                1    3.6   
...                                                     ...    ...   
rymdkapsel                                                1    1.1   
sZone-Online                                             58   56.7   
the static speaks my name                                 8    2.0   
theHunter                                               124  309.2   
theHunter Primal                                          4   85.9   

                                                    Percent Play  
Game                                                              
007 Legends                                             0.000088  
0RBITALIS                                               0.000264  
1... 2... 3... KICK IT! (Drop That Beat Like an...      0.000441  
10 Second Ninja                                         0.000176  
10,000,000                                              0.000088  
...                                                          ...  
rymdkapsel                                              0.000088  
sZone-Online                                            0.005110  
the static speaks my name                               0.000705  
theHunter                                               0.010925  
theHunter Primal                                        0.000352  

[3600 rows x 3 columns]

In [5]:
# file 2 cleaning (should drop more columns after discussing)
sales_data=sales_data.loc[(sales_data["Platform"])=="PC",:]
sales_data=sales_data.rename(columns={"Name": "Game"})
sales_data

Rank                                   Game         Genre ESRB_Rating  \
3          4          PlayerUnknown's Battlegrounds       Shooter         NaN   
9         10                              Minecraft          Misc         NaN   
22        23                            Garry's Mod          Misc         NaN   
36        37                  Counter-Strike Source       Shooter           M   
55        56                               Portal 2       Shooter         E10   
...      ...                                    ...           ...         ...   
55779  55780  Zoo Tycoon Ultimate Animal Collection    Simulation           E   
55782  55783                             Ion Maiden       Shooter         NaN   
55786  55787                  In the Valley of Gods     Adventure         NaN   
55787  55788                            Indivisible  Role-Playing         NaN   
55788  55789                             Lost Ember     Adventure          RP   

      Platform         Publisher              Developer  Critic_Score  \
3           PC  PUBG Corporation       PUBG Corporation           NaN   
9           PC            Mojang              Mojang AB          10.0   
22          PC           Unknown             Team Garry           NaN   
36          PC          VU Games         Valve Software           9.3   
55          PC             Valve         Valve Software           9.7   
...        ...               ...                    ...           ...   
55779       PC        THQ Nordic  Frontier Developments           NaN   
55782       PC         3D Realms             Voidpoint            NaN   
55786       PC      Campo Santo            Campo Santo            NaN   
55787       PC         505 Games         Lab Zero Games           NaN   
55788       PC   Mooneye Studios        Mooneye Studios           NaN   

       User_Score  Total_Shipped  Global_Sales  NA_Sales  PAL_Sales  JP_Sales  \
3             NaN          36.60           NaN       NaN        NaN       NaN   
9             NaN          30.01           NaN       NaN        NaN       NaN   
22            NaN          18.58           NaN       NaN        NaN       NaN   
36            NaN          15.00           NaN       NaN        NaN       NaN   
55            NaN          13.06           NaN       NaN        NaN       NaN   
...           ...            ...           ...       ...        ...       ...   
55779         NaN            NaN           NaN       NaN        NaN       NaN   
55782         NaN            NaN           NaN       NaN        NaN       NaN   
55786         NaN            NaN           NaN       NaN        NaN       NaN   
55787         NaN            NaN           NaN       NaN        NaN       NaN   
55788         NaN            NaN           NaN       NaN        NaN       NaN   

       Other_Sales    Year  
3              NaN  2017.0  
9              NaN  2010.0  
22             NaN  2004.0  
36             NaN  2004.0  
55             NaN  2011.0  
...            ...     ...  
55779          NaN  2018.0  
55782          NaN  2019.0  
55786          NaN  2019.0  
55787          NaN  2019.0  
55788          NaN  2019.0  

[10978 rows x 16 columns]

In [6]:
# merging files on game (need to drop rows with NaN value on players / hours)
steam_sales_data = pd.merge(group_steam,sales_data,on='Game',how='outer')

In [7]:
# change all NaN values in 'platform' column to "PC" - all steam data is PC data
steam_sales_data.Platform = steam_sales_data.Platform.fillna('PC')

# drop rows with NaN values in players / hours
steam_sales_data.dropna(subset = ["Players", "Hours"], inplace = True)

# drop Game duplicates
steam_sales_data.drop_duplicates(subset = ["Game"], inplace = True)

steam_sales_data=steam_sales_data.loc[(steam_sales_data["Year"])>=1980,:]


In [8]:
steam_sales_data

Game  Players  Hours  \
0                                           007 Legends      1.0    0.7   
2     1... 2... 3... KICK IT! (Drop That Beat Like a...      5.0   20.0   
4                                            10,000,000      1.0    3.6   
7                                12 Labours of Hercules      7.0   34.7   
8             12 Labours of Hercules II The Cretan Bull      8.0   34.3   
...                                                 ...      ...    ...   
3585                                     bit Dungeon II      2.0   10.5   
3592                                    iBomber Defense      2.0   11.1   
3593                            iBomber Defense Pacific      4.0   17.9   
3600                                            rFactor      1.0    2.2   
3609                                   theHunter Primal      4.0   85.9   

      Percent Play     Rank     Genre ESRB_Rating Platform  \
0         0.000088  20216.0   Shooter           T       PC   
2         0.000441   9199.0      Misc         NaN       PC   
4         0.000088  52595.0    Puzzle         NaN       PC   
7         0.000617   9863.0  Strategy         NaN       PC   
8         0.000705   9439.0  Strategy         NaN       PC   
...            ...      ...       ...         ...      ...   
3585      0.000176  53100.0    Action         NaN       PC   
3592      0.000176  33961.0  Strategy         NaN       PC   
3593      0.000352  12305.0  Strategy         NaN       PC   
3600      0.000088  20750.0    Racing         NaN       PC   
3609      0.000352   8618.0    Action         NaN       PC   

                     Publisher         Developer  Critic_Score  User_Score  \
0                   Activision           Eurocom           NaN         NaN   
2               Dejobaan Games    Dejobaan Games           NaN         NaN   
4                      Unknown  EightyEightGames           NaN         NaN   
7              Jetdogs Studios   Jetdogs Studios           NaN         NaN   
8                      Unknown   Jetdogs Studios           NaN         NaN   
...                        ...               ...           ...         ...   
3585                   Unknown   Kinto Games LLC           NaN         NaN   
3592                 Chillingo      Cobra Mobile           NaN         NaN   
3593              Cobra Mobile      Cobra Mobile           NaN         NaN   
3600  Image Space Incorporated  Image Space Inc.           NaN         NaN   
3609         Avalanche Studios  Expansive Worlds           NaN         NaN   

      Total_Shipped  Global_Sales  NA_Sales  PAL_Sales  JP_Sales  Other_Sales  \
0               NaN           0.0       NaN        0.0       NaN          0.0   
2              0.19           NaN       NaN        NaN       NaN          NaN   
4               NaN           NaN       NaN        NaN       NaN          NaN   
7              0.16           NaN       NaN        NaN       NaN          NaN   
8              0.18           NaN       NaN        NaN       NaN          NaN   
...             ...           ...       ...        ...       ...          ...   
3585            NaN           NaN       NaN        NaN       NaN          NaN   
3592            NaN           NaN       NaN        NaN       NaN          NaN   
3593           0.10           NaN       NaN        NaN       NaN          NaN   
3600            NaN           0.0       NaN        0.0       NaN          0.0   
3609           0.21           NaN       NaN        NaN       NaN          NaN   

        Year  
0     2012.0  
2     2011.0  
4     2018.0  
7     2015.0  
8     2005.0  
...      ...  
3585  2018.0  
3592  2011.0  
3593  2012.0  
3600  2005.0  
3609  2015.0  

[1790 rows x 19 columns]

In [9]:
steam_sales_data.to_csv('resources/steam_sales_data.csv')

In [10]:
steam_sales_data.insert(4,"Average_Hours", (steam_sales_data["Hours"]/steam_sales_data["Players"]),True)

In [11]:
steam_sales_data["Average_Hours"].mean()

15.028856068014623

In [12]:
steam_sales_data["Players"].mean()

28.188826815642457

In [13]:
steam_sales_data[["Players", "Average_Hours"]].groupby("Players").mean()

Average_Hours
Players               
1.0          12.494355
2.0          11.611813
3.0           7.502299
4.0           9.057174
5.0          10.809620
...                ...
801.0        41.943446
1069.0       15.057437
1377.0      234.402033
2323.0       74.762505
4841.0      202.785499

[174 rows x 1 columns]